In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import pandas as pd

In [2]:
url = 'https://docms.gov.taipei/News.aspx?n=03317D8229965B64&sms=87415A8B9CE81B16&page=1&PageSize=200'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
questions = soup.find_all('td', class_='CCMS_jGridView_td_Class_1')
df_result= pd.DataFrame(columns=['Time', 'Url', 'Question', 'Answer', '點閱數', '資料更新', '資料檢視', '資料維護'])

In [3]:
question_list = []
x = 0

for question in questions:
    question_text = questions[x].find('a').text
    x = x + 1
    question_list.append(question_text)

df_result['Question'] = question_list

In [4]:
url_list = []

for x in questions:
    url = 'https://docms.gov.taipei/' + x.find('a')['href']
    url_list.append(url)

df_result['Url'] = url_list

In [6]:
import re
answer_list = []
current_time_list = []
click_list = []
data_update_list = []
data_review_list = []
data_maintainence_list = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    answers = soup.find_all('div', class_='p')
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for tag in answers:
        answers = tag.text
    
    clicks = soup.find('span', {'id': 'hitcount'})
    click_value = clicks.text.strip() if clicks else None
    click_numeric = int(re.search(r'\d+', click_value).group()) if click_value else None

    data_update = soup.find_all('li', {'data-index': '2'})
    for li in data_update:
        data_update_value = li.find('span')
        if data_update_value and '資料更新' in data_update_value.text:
            update_info = data_update_value.text.strip().replace('資料更新：', '')
            break

    data_review = soup.find_all('li', {'data-index': '3'})
    for li in data_review:
        data_review_value = li.find('span')
        if data_review_value and '資料檢視' in data_review_value.text:
            review_info = data_review_value.text.strip().replace('資料檢視：', '')
            break

    data_maintainence = soup.find_all('li', {'data-index': '4'})
    for li in data_maintainence:
        data_maintainence_value = li.find('span')
        if data_maintainence_value and '資料維護' in data_maintainence_value.text:
            maintainence_info = data_maintainence_value.text.strip().replace('資料維護：', '')
            break

    answer_list.append(answers)
    current_time_list.append(current_time)
    click_list.append(click_numeric)
    data_update_list.append(update_info)
    data_review_list.append(review_info)
    data_maintainence_list.append(maintainence_info)

df_result['Time'] = current_time_list
df_result['Answer'] = answer_list
df_result['點閱數'] = click_list
df_result['資料更新'] = data_update_list
df_result['資料檢視'] = data_review_list
df_result['資料維護'] = data_maintainence_list

In [7]:
df_result.to_csv('兵役局QA.csv', index=False)